## Questions to answer !
Which factor influenced a candidate in getting placed?<br>
Does percentage matters for one to get placed?<br>
Which degree specialization is much demanded by corporate?<br>
Play with the data conducting all statistical tests.<br>

In [11]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Read data

In [3]:
data = pd.read_csv('Placement_Data_Full_Class.csv')

In [4]:
target_cols = ['status', 'salary']
predictor_cols = list(set(data.columns).difference(set(target_cols)))
character_cols = ['gender', 'ssc_b', 'hsc_b', 'hsc_s', 'degree_t', 'workex', 'specialisation', 'status']
numerical_cols = ['sl_no', 'ssc_p', 'hsc_p', 'degree_p', 'etest_p', 'mba_p', 'salary']
continuous_cols = ['ssc_p', 'hsc_p', 'degree_p', 'etest_p', 'mba_p', 'salary']
nominal_cols = ['gender', 'ssc_b', 'hsc_b', 'hsc_s', 'degree_t', 'workex', 'specialisation', 'status']
ordinal_cols = []

In [5]:
data[numerical_cols].describe()

,sl_no,ssc_p,hsc_p,degree_p,etest_p,mba_p,salary
count,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,148.000000
mean,108.000000,67.303395,66.333163,66.370186,72.100558,62.278186,288655.405405
std,62.209324,10.827205,10.897509,7.358743,13.275956,5.833385,93457.452420
min,1.000000,40.890000,37.000000,50.000000,50.000000,51.210000,200000.000000
25%,54.500000,60.600000,60.900000,61.000000,60.000000,57.945000,240000.000000
50%,108.000000,67.000000,65.000000,66.000000,71.000000,62.000000,265000.000000
75%,161.500000,75.700000,73.000000,72.000000,83.500000,66.255000,300000.000000
max,215.000000,89.400000,97.700000,91.000000,98.000000,77.890000,940000.000000


### UNIVARAIATE ANALYSIS

In [6]:
data.salary.skew()

3.5697471998711054

In [7]:
data.salary.describe()

count       148.000000
mean     288655.405405
std       93457.452420
min      200000.000000
25%      240000.000000
50%      265000.000000
75%      300000.000000
max      940000.000000
Name: salary, dtype: float64

In [38]:
fig = make_subplots(rows=1, 
                    cols=2,
                    subplot_titles=['Salary distribution', 'Salary range'])

fig.add_trace(go.Histogram(x=data['salary'], 
                           name='Histogram'),
              row=1,
              col=1)

fig.add_trace(go.Box(y=data['salary'],
                     boxpoints='all',
                     name='boxplot'),
              row=1,
              col=2)

# # Add shapes
# fig.add_shape(
#         # Line Vertical
#         dict(
#             type="line",
#             x0=data.salary.mean(),
#             x1=data.salary.mean(),
#             y0=0,
#             y1=100,
#             line=dict(
#                 color="RoyalBlue",
#                 width=3
#             )
# ))